In [1]:
import numpy as np
import mpmath 
import matplotlib.pyplot as plt
from IPython.display import Audio
import math as mth
from time import time
from contextlib import contextmanager
from scipy import linalg, signal, constants, optimize

In [2]:
if False:
    data = np.loadtxt('rho_2dots_dt5e-5_fix.dat')
    rho_00_dot1 = data[:,0]
    rho_01_dot1 = data[:,1] + 1j*data[:,2]

    rho_00_dot2 = data[:,4]
    rho_01_dot2 = data[:,5] + 1j*data[:,6]
    data = np.loadtxt('rho_2dots_dt5e-5_fix_nint.dat')
    rho_00_dot1_nint = data[:,0]
    rho_01_dot1_nint = data[:,1] + 1j*data[:,2]

    rho_00_dot2_nint = data[:,4]
    rho_01_dot2_nint = data[:,5] + 1j*data[:,6]
    data = np.loadtxt('chi_2dots_fix.dat')
    chi_rad_elliot = data[:,0]
    
    np.savez('Elliot_2dot_data.npz',rho_00_dot1=rho_00_dot1,
        rho_01_dot1=rho_01_dot1,
        rho_00_dot2=rho_00_dot2,
        rho_01_dot2=rho_01_dot2,
        rho_00_dot1_nint=rho_00_dot1_nint,
        rho_01_dot1_nint=rho_01_dot1_nint,
        rho_00_dot2_nint=rho_00_dot2_nint,
        rho_01_dot2_nint=rho_01_dot2_nint,
        chi_rad_elliot=chi_rad_elliot)

In [3]:
data = np.load('Elliot_2dot_data.npz')
rho_00_dot1 = data['rho_00_dot1']
rho_01_dot1 = data['rho_01_dot1']
rho_00_dot2 = data['rho_00_dot2']
rho_01_dot2 = data['rho_01_dot2']
rho_00_dot1_nint = data['rho_00_dot1_nint']
rho_01_dot1_nint = data['rho_01_dot1_nint']
rho_00_dot2_nint = data['rho_00_dot2_nint']
rho_01_dot2_nint = data['rho_01_dot2_nint']
chi_rad_elliot = data['chi_rad_elliot']

In [4]:
@contextmanager
def timer(prefix=None):
    t_start = time()
    yield
    t_end = time()
    t = t_end - t_start
    
    minutes = np.floor(t / 60)
    seconds = t % 60
    if prefix is None:
        print(minutes,'minutes',seconds,'seconds')
        
    else:
        print(prefix+':',minutes,'minutes',seconds,'seconds')

In [5]:
def get_uvw(rho_00,rho_01):
    u = 2*rho_01.real
    v = 2*rho_01.imag
    w = 1-2*rho_00
    return u,v,w

In [6]:
u1_elliot,v1_elliot,w1_elliot = get_uvw(rho_00_dot1,rho_01_dot1)
u2_elliot,v2_elliot,w2_elliot = get_uvw(rho_00_dot2,rho_01_dot2)

u1_elliot_nint,v1_elliot_nint,w1_elliot_nint = get_uvw(rho_00_dot1_nint,rho_01_dot1_nint)
u2_elliot_nint,v2_elliot_nint,w2_elliot_nint = get_uvw(rho_00_dot2_nint,rho_01_dot2_nint)

t_elliot = np.linspace(0,10,u1_elliot.size)

In [7]:
def compare_frequency_spectra_solutions(signal_t_1,signal_t_2,t,minimum_f=None,maximum_f=None,sol=' Guess',vline=False):
    #plots the freq spec for a given signal.  minimum_f and maximum_f must be in Hz. signal is the variable name
    stop_time = t[-1]
    dt = stop_time/t.size
    
    Signal_w_1 = np.fft.fftshift(np.fft.fft(signal_t_1))
    Signal_w_2 = np.fft.fftshift(np.fft.fft(signal_t_2))
    freq = np.fft.fftshift(np.fft.fftfreq(signal_t_1.size,dt))

   
    plt.figure()
    plt.semilogy(freq,abs(Signal_w_1),label='Chad'+sol)
    plt.semilogy(freq,abs(Signal_w_2),label='Elliot Solution')
    if vline == True:
        for frequency in omega_k/(2*np.pi):
            plt.axvline(frequency,c='k')
    plt.legend()
    plt.grid()
    plt.xlabel('Linear Frequency')
    plt.xlim(minimum_f,maximum_f)
    plt.ylabel('|Magnitude|')

    plt.figure()
    plt.plot(freq,np.angle(Signal_w_1,deg=True),label='Chad'+sol)
    plt.plot(freq,np.angle(Signal_w_2,deg=True),label='Elliots Solution')
    if vline == True:
        for frequency in omega_k/(2*np.pi):
            plt.axvline(frequency,c='k')
    plt.ylabel('Phase (degrees)')
    plt.xlabel('Linear Frequency')
    plt.xlim(minimum_f,maximum_f)
    plt.legend()

In [8]:
#Define Constants

#General Parameters
num_dots = 2
omega_0 = 2278.9013 # in rad/picosecond
period = 2*np.pi/omega_0
f0 = omega_0 / (2*np.pi)

stop_time = 10 #in picoseconds

#Dot parameters
sigma = 2278.9013
t0 = 0.0
hbar = 0.65821193
T1 = 1e6
T2 = 2e6
x = np.array([0.0,0.0])
y = np.array([-0.00175,0.00175])

R = y[1] - y[0]
#Misc Parameters
x_hat = np.array([1,0,0])
y_hat = np.array([0,1,0])
r_hat_dot1 = np.array([0,1,0])
r_hat_dot2 = np.array([0,-1,0])

c0 = 299.792458 #um/ps
mu0 = 2.0133545e-04
eps0 = 1/(mu0*c0**2)
print(eps0)
d = 5.2917721e-4
d_vector = d*x_hat
E0 = 1558.8929

#omega_r = d*E0/hbar #+ E_rad*5.2917721e-4/hbar 
f_r = 2.2669
omega_r = 2*np.pi*f_r

0.05526349463314177


In [9]:
delta_f = 0.45
delta_omega = 2*np.pi*delta_f
B = 4
C = 4
i = np.arange(B+1)
j = np.arange(-B,B+1)
k = np.arange(-C,C+1)

i,j,k = np.meshgrid(i,j,k,indexing='ij')
ij_sum_abs = abs(i) + abs(j)
ij = ij_sum_abs<=B
i = i[ij]
j = j[ij]
k = k[ij]

omega_k = i * omega_0 + j * omega_r + k * delta_omega
omega_k = np.unique(omega_k)

#Remove negative frequencies for real signals 
omega_k = omega_k[omega_k>=0]
omega_k = np.sort(omega_k)
omega_k = np.hstack((omega_k,-omega_k[1:]))
omega_k = np.unique(omega_k)

num_frequencies = omega_k.size
num_coefficients = num_frequencies
num_unknowns = num_dots*3*num_coefficients
degrees_of_freedom_remaining = int(num_unknowns - 3*num_dots)
num_tests = int(degrees_of_freedom_remaining / (3*num_dots))

print('Solving',num_frequencies,'frequencies')
print('Solving',num_coefficients,'coefficients per signal')
print('Solving',num_unknowns,'degrees of freedom')

Solving 369 frequencies
Solving 369 coefficients per signal
Solving 2214 degrees of freedom


In [10]:
wave_vector = x_hat
r = x_hat*0.0
args = np.dot(wave_vector,r) - omega_0*(t_elliot-t0)

#pulse = np.exp(omega_0*(t - t0)**2/(2*sigma**2))
#pulse = 0
pulse = 1
El = (E0*np.cos(args)*pulse)*x_hat[:,np.newaxis]

chi_elliot = chi_rad_elliot + np.dot(d_vector,El) / hbar

In [11]:
def get_gamma(t):
    #Creates a matrix of trigonometric basis functions
    return np.exp(1j*omega_k*t[:,np.newaxis])

In [12]:
def get_gamma_t0(t0):
    #Creates a matrix of trigonometric basis functions evaluated at an instant in time, t0
    #Used for enforcing initial conditions in the Newton Solver
    return np.exp(1j*omega_k*t0)

In [13]:
def get_gamma_prime(t):
    #Creates a matrix of derivatives of trig basis functions
    return 1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])

In [14]:
def get_gamma_double_prime(t):
    #Second derivative matrix
    return -omega_k**2*np.exp(1j*omega_k*t[:,np.newaxis])

In [15]:
def get_rhos(u,v,w):
    rho_00 = 0.5 - 0.5*w
    rho_01 = 0.5*u + 0.5j*v
    return rho_00, rho_01

In [16]:
def get_P(u_coeff,v_coeff,w_coeff,t):
    #Evaluates the polarization
    delay = t - (R/c0)
    gamma_delay = get_gamma(delay)
    
    u_delay = gamma_delay@u_coeff
    v_delay = gamma_delay@v_coeff
    w_delay = gamma_delay@w_coeff
    
    rho_00_delay,rho_01_delay = get_rhos(u_delay,v_delay,w_delay)
    
    P = 2*(rho_01_delay.real)*d_vector[:,np.newaxis]
    
    return P

In [17]:
def get_P_prime(u_coeff,v_coeff,w_coeff,t):
    delay = t - (R/c0)
    gamma_prime_delay = get_gamma_prime(delay)
    u_prime_delay = gamma_prime_delay@u_coeff
    v_prime_delay = gamma_prime_delay@v_coeff
    w_prime_delay = gamma_prime_delay@w_coeff
    
    rho_00_prime_delay,rho_01_prime_delay = get_rhos(u_prime_delay,v_prime_delay,w_prime_delay)
    P_prime = 2*(rho_01_prime_delay.real)*d_vector[:,np.newaxis]
    
    return P_prime

In [18]:
def get_P_doubleprime(u_coeff,v_coeff,w_coeff,t):
    delay = t - (R/c0)
    gamma_doubleprime = get_gamma_double_prime(delay)
    u_doubleprime_delay = gamma_doubleprime@u_coeff
    v_doubleprime_delay = gamma_doubleprime@v_coeff
    w_doubleprime_delay = gamma_doubleprime@w_coeff
    
    rho_00_doubleprime_delay,rho_01_doubleprime_delay = get_rhos(u_doubleprime_delay,v_doubleprime_delay,w_doubleprime_delay)
    P_doubleprime = 2*(rho_01_doubleprime_delay.real)*d_vector[:,np.newaxis]
    
    return P_doubleprime

In [19]:
def get_E_rad(u_coeff,v_coeff,w_coeff,t,r_hat):
    #Evaluates the radiation field from one dot onto another.  In the 2-dot case R is a constant for both dots.
    P = get_P(u_coeff,v_coeff,w_coeff,t)
    P_prime = get_P_prime(u_coeff,v_coeff,w_coeff,t)
    P_doubleprime = get_P_doubleprime(u_coeff,v_coeff,w_coeff,t)
    
    I = np.eye(3)
    
    E_rad = np.dot(I - np.outer(r_hat,r_hat),P_doubleprime/(c0**2*R))
    
    E_rad += np.dot(I-3*np.outer(r_hat,r_hat),P_prime/(c0*R**2))

    E_rad += np.dot(I-3*np.outer(r_hat,r_hat),P/(R**3))
    
    E_rad *= -1/(4*np.pi*eps0)
    
    return E_rad

In [20]:
def get_chi(u_coeff,v_coeff,w_coeff,t,r_hat):
    #Takes in a radiation field from a specific dot and returns chi for that dot.
    wave_vector = x_hat
    r = x_hat*0.0
    args = np.dot(wave_vector,r) - omega_0*(t-t0)
    
    #pulse = np.exp(omega_0*(t - t0)**2/(2*sigma**2))
    #pulse = 0
    pulse = 1
    El = (E0*np.cos(args)*pulse)*x_hat[:,np.newaxis]
    Er = get_E_rad(u_coeff,v_coeff,w_coeff,t,r_hat)
    
    E = El + Er
    chi = np.dot(d_vector,E) / hbar
    
    return chi

In [21]:
def get_dE_rad_dDu(r_hat,t):
    I = np.eye(3)
    t_delay = t - (R/c0)
    
    dP_dDu = np.exp(1j*omega_k*t_delay[:,np.newaxis])*d_vector[:,np.newaxis,np.newaxis]/R**3
    dP_prime_dDu = 1j*omega_k*np.exp(1j*omega_k*t_delay[:,np.newaxis])*d_vector[:,np.newaxis,np.newaxis]/(c0*R**2)
    dP_doubleprime_dDu = -omega_k**2*np.exp(1j*omega_k*t_delay[:,np.newaxis])*d_vector[:,np.newaxis,np.newaxis]/(c0**2*R)
    
    dE_rad_dDu = np.dot((I - np.outer(r_hat,r_hat)),np.transpose(dP_doubleprime_dDu,axes=(1,0,2)))
    dE_rad_dDu += np.dot((I-3*np.outer(r_hat,r_hat)),np.transpose(dP_prime_dDu,axes=(1,0,2)))
    dE_rad_dDu += np.dot((I-3*np.outer(r_hat,r_hat)),np.transpose(dP_dDu,axes=(1,0,2)))
    
    dE_rad_dDu = np.transpose(dE_rad_dDu,axes=(1,0,2))
                  
    return dE_rad_dDu

In [22]:
def get_dChi_dDu(u_coeff,v_coeff,w_coeff,t,r_hat):
    dE_rad_dDu = get_dE_rad_dDu(r_hat,t)
    
    dChi_dDu = np.dot(d_vector,dE_rad_dDu)
    
    return dChi_dDu

In [23]:
t_k = np.linspace(0,stop_time,num_coefficients - 1)

In [24]:
def get_Jacobian(coefficients,t):
    u1_coeff = coefficients[:num_coefficients]
    v1_coeff = coefficients[num_coefficients:2*num_coefficients]
    w1_coeff = coefficients[2*num_coefficients:3*num_coefficients]
    
    u2_coeff = coefficients[3*num_coefficients:4*num_coefficients]
    v2_coeff = coefficients[4*num_coefficients:5*num_coefficients]
    w2_coeff = coefficients[5*num_coefficients:]
    
    gamma_t = get_gamma(t)
    
    u1 = gamma_t@u1_coeff
    v1 = gamma_t@v1_coeff
    w1 = gamma_t@w1_coeff
    
    u2 = gamma_t@u2_coeff
    v2 = gamma_t@v2_coeff
    w2 = gamma_t@w2_coeff
    
    dChi1_dDu = get_dChi_dDu(u2_coeff,v2_coeff,w2_coeff,t,r_hat_dot2)
    dChi2_dDu = get_dChi_dDu(u1_coeff,v1_coeff,w1_coeff,t,r_hat_dot1)

    chi1 = get_chi(u2_coeff,v2_coeff,w2_coeff,t,r_hat_dot2)[:,np.newaxis]
    chi2 = get_chi(u1_coeff,v1_coeff,w1_coeff,t,r_hat_dot1)[:,np.newaxis]
    
    df1_dot1_dDu1 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    df1_dot1_dDv1 = -omega_0*np.exp(1j*omega_k*t[:,np.newaxis])
    df1_dot1_dDw1 = np.zeros_like(df1_dot1_dDv1)
    df1_dot1_dDu2 = np.zeros_like(df1_dot1_dDv1)
    df1_dot1_dDv2 = np.zeros_like(df1_dot1_dDv1)
    df1_dot1_dDw2 = np.zeros_like(df1_dot1_dDv1)
    grad_f1_dot1 = np.hstack((df1_dot1_dDu1,df1_dot1_dDv1,df1_dot1_dDw1,df1_dot1_dDu2,df1_dot1_dDv2,df1_dot1_dDw2))
    
    df2_dot1_dDu1 = omega_0*np.exp(1j*omega_k*t[:,np.newaxis]) - 2*dChi1_dDu*w1[:,np.newaxis]
    df2_dot1_dDv1 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    df2_dot1_dDw1 = -2*chi1*np.exp(1j*omega_k*t[:,np.newaxis])
    df2_dot1_dDu2 = np.zeros_like(df1_dot1_dDv1)
    df2_dot1_dDv2 = np.zeros_like(df1_dot1_dDv1)
    df2_dot1_dDw2 = np.zeros_like(df1_dot1_dDv1)
    grad_f2_dot1 = np.hstack((df2_dot1_dDu1,df2_dot1_dDv1,df2_dot1_dDw1,df2_dot1_dDu2,df2_dot1_dDv2,df2_dot1_dDw2))
    
    df3_dot1_dDu1 = 2*v1[:,np.newaxis]*dChi1_dDu
    df3_dot1_dDv1 = 2*chi1*np.exp(1j*omega_k*t[:,np.newaxis])
    df3_dot1_dDw1 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    df3_dot1_dDu2 = np.zeros_like(df1_dot1_dDv1)
    df3_dot1_dDv2 = np.zeros_like(df1_dot1_dDv1)
    df3_dot1_dDw2 = np.zeros_like(df1_dot1_dDv1)
    grad_f3_dot1 = np.hstack((df3_dot1_dDu1,df3_dot1_dDv1,df3_dot1_dDw1,df3_dot1_dDu2,df3_dot1_dDv2,df3_dot1_dDw2))
    
    df4_dot1_dDu1 = np.ones(omega_k.size)[:,np.newaxis].T
    df4_dot1_dDv1 = np.zeros_like(df4_dot1_dDu1)
    df4_dot1_dDw1 = np.zeros_like(df4_dot1_dDu1)
    df4_dot1_dDu2 = np.zeros_like(df4_dot1_dDu1)
    df4_dot1_dDv2 = np.zeros_like(df4_dot1_dDu1)
    df4_dot1_dDw2 = np.zeros_like(df4_dot1_dDu1)
    grad_f4_dot1 = np.hstack((df4_dot1_dDu1,df4_dot1_dDv1,df4_dot1_dDw1,df4_dot1_dDu2,df4_dot1_dDv2,df4_dot1_dDw2))
    
    df5_dot1_dDu1 = np.zeros_like(df4_dot1_dDu1)
    df5_dot1_dDv1 = np.ones_like(df4_dot1_dDu1)
    df5_dot1_dDw1 = np.zeros_like(df4_dot1_dDu1)
    df5_dot1_dDu2 = np.zeros_like(df4_dot1_dDu1)
    df5_dot1_dDv2 = np.zeros_like(df4_dot1_dDu1)
    df5_dot1_dDw2 = np.zeros_like(df4_dot1_dDu1)
    grad_f5_dot1 = np.hstack((df5_dot1_dDu1,df5_dot1_dDv1,df5_dot1_dDw1,df5_dot1_dDu2,df5_dot1_dDv2,df5_dot1_dDw2))
    
    df6_dot1_dDu1 = np.zeros_like(df4_dot1_dDu1)
    df6_dot1_dDv1 = np.zeros_like(df4_dot1_dDv1)
    df6_dot1_dDw1 = np.ones_like(df4_dot1_dDw1)
    df6_dot1_dDu2 = np.zeros_like(df4_dot1_dDu1)
    df6_dot1_dDv2 = np.zeros_like(df4_dot1_dDu1)
    df6_dot1_dDw2 = np.zeros_like(df4_dot1_dDu1)
    grad_f6_dot1 = np.hstack((df6_dot1_dDu1,df6_dot1_dDv1,df6_dot1_dDw1,df6_dot1_dDu2,df6_dot1_dDv2,df6_dot1_dDw2))
    
    df1_dot2_dDu1 = np.zeros_like(df1_dot1_dDv1)
    df1_dot2_dDv1 = np.zeros_like(df1_dot1_dDv1)
    df1_dot2_dDw1 = np.zeros_like(df1_dot1_dDv1)
    df1_dot2_dDu2 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    df1_dot2_dDv2 = -omega_0*np.exp(1j*omega_k*t[:,np.newaxis])
    df1_dot2_dDw2 = np.zeros_like(df1_dot2_dDv1)
    grad_f1_dot2 = np.hstack((df1_dot2_dDu1,df1_dot2_dDv1,df1_dot2_dDw1,df1_dot2_dDu2,df1_dot2_dDv2,df1_dot2_dDw2))
    
    df2_dot2_dDu1 = np.zeros_like(df1_dot1_dDv1)
    df2_dot2_dDv1 = np.zeros_like(df1_dot1_dDv1)
    df2_dot2_dDw1 = np.zeros_like(df1_dot1_dDv1)
    df2_dot2_dDu2 = omega_0*np.exp(1j*omega_k*t[:,np.newaxis]) - 2*dChi2_dDu*w2[:,np.newaxis]
    df2_dot2_dDv2 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    df2_dot2_dDw2 = -2*chi2*np.exp(1j*omega_k*t[:,np.newaxis])
    grad_f2_dot2 = np.hstack((df2_dot2_dDu1,df2_dot2_dDv1,df2_dot2_dDw1,df2_dot2_dDu2,df2_dot2_dDv2,df2_dot2_dDw2))
    
    df3_dot2_dDu1 = np.zeros_like(df1_dot1_dDv1)
    df3_dot2_dDv1 = np.zeros_like(df1_dot1_dDv1)
    df3_dot2_dDw1 = np.zeros_like(df1_dot1_dDv1)
    df3_dot2_dDu2 = 2*v2[:,np.newaxis]*dChi2_dDu
    df3_dot2_dDv2 = 2*chi2*np.exp(1j*omega_k*t[:,np.newaxis])
    df3_dot2_dDw2 = -1j*omega_k*np.exp(1j*omega_k*t[:,np.newaxis])
    grad_f3_dot2 = np.hstack((df3_dot2_dDu1,df3_dot2_dDv1,df3_dot2_dDw1,df3_dot2_dDu2,df3_dot2_dDv2,df3_dot2_dDw2))
    
    df4_dot2_dDu1 = np.zeros_like(df4_dot1_dDv1)
    df4_dot2_dDv1 = np.zeros_like(df4_dot1_dDv1)
    df4_dot2_dDw1 = np.zeros_like(df4_dot1_dDv1)
    df4_dot2_dDu2 = np.ones(omega_k.size)[:,np.newaxis].T
    df4_dot2_dDv2 = np.zeros_like(df4_dot2_dDu1)
    df4_dot2_dDw2 = np.zeros_like(df4_dot2_dDu1)
    grad_f4_dot2 = np.hstack((df4_dot2_dDu1,df4_dot2_dDv1,df4_dot2_dDw1,df4_dot2_dDu2,df4_dot2_dDv2,df4_dot2_dDw2))

    df5_dot2_dDu1 = np.zeros_like(df5_dot1_dDv1)
    df5_dot2_dDv1 = np.zeros_like(df5_dot1_dDv1)
    df5_dot2_dDw1 = np.zeros_like(df5_dot1_dDv1)
    df5_dot2_dDu2 = np.zeros_like(df5_dot1_dDu1)
    df5_dot2_dDv2 = np.ones_like(df5_dot1_dDu1)
    df5_dot2_dDw2 = np.zeros_like(df5_dot1_dDu1)
    grad_f5_dot2 = np.hstack((df5_dot2_dDu1,df5_dot2_dDv1,df5_dot2_dDw1,df5_dot2_dDu2,df5_dot2_dDv2,df5_dot2_dDw2))
    
    df6_dot2_dDu1 = np.zeros_like(df6_dot1_dDv1)
    df6_dot2_dDv1 = np.zeros_like(df6_dot1_dDv1)
    df6_dot2_dDw1 = np.zeros_like(df6_dot1_dDv1)
    df6_dot2_dDu2 = np.zeros_like(df6_dot1_dDu1)
    df6_dot2_dDv2 = np.zeros_like(df6_dot1_dDv1)
    df6_dot2_dDw2 = np.ones_like(df6_dot1_dDw1)
    grad_f6_dot2 = np.hstack((df6_dot2_dDu1,df6_dot2_dDv1,df6_dot2_dDw1,df6_dot2_dDu2,df6_dot2_dDv2,df6_dot2_dDw2))
    
    Jacobian_dot1 = np.vstack((grad_f1_dot1,grad_f2_dot1,grad_f3_dot1,grad_f4_dot1,grad_f5_dot1,grad_f6_dot1))
    Jacobian_dot2 = np.vstack((grad_f1_dot2,grad_f2_dot2,grad_f3_dot2,grad_f4_dot2,grad_f5_dot2,grad_f6_dot2))
    
    Jacobian = np.vstack((Jacobian_dot1,Jacobian_dot2))
    return Jacobian

In [25]:
def plot_error(t,f1,f2,f3,f4,f5,f6):
    plt.figure()
    plt.semilogy(t,abs(f1),label='f1')
    plt.semilogy(t,abs(f2),label='f2')
    plt.semilogy(t,abs(f3),label='f3')
    plt.legend()
    plt.show()

In [26]:
def get_f(coefficients,t=t_k):
    #Takes in an array of Fourier coefficients for u,v,w for both dots and returns the error functions in time
    u1_coeff = coefficients[:num_coefficients]
    v1_coeff = coefficients[num_coefficients:2*num_coefficients]
    w1_coeff = coefficients[2*num_coefficients:3*num_coefficients]
    
    u2_coeff = coefficients[3*num_coefficients:4*num_coefficients]
    v2_coeff = coefficients[4*num_coefficients:5*num_coefficients]
    w2_coeff = coefficients[5*num_coefficients:]

    gamma_t = get_gamma(t)
    gamma_prime = get_gamma_prime(t)
    gamma_0 = get_gamma_t0(0)
    
    chi1 = get_chi(u2_coeff,v2_coeff,w2_coeff,t,r_hat_dot2)
    chi2 = get_chi(u1_coeff,v1_coeff,w1_coeff,t,r_hat_dot1)
    
    u1 = gamma_t@u1_coeff
    v1 = gamma_t@v1_coeff
    w1 = gamma_t@w1_coeff
    
    u2 = gamma_t@u2_coeff
    v2 = gamma_t@v2_coeff
    w2 = gamma_t@w2_coeff
    
    u1_prime = gamma_prime@u1_coeff
    v1_prime = gamma_prime@v1_coeff
    w1_prime = gamma_prime@w1_coeff
    
    u2_prime = gamma_prime@u2_coeff
    v2_prime = gamma_prime@v2_coeff
    w2_prime = gamma_prime@w2_coeff
    
    u1_0 = gamma_0@u1_coeff
    v1_0 = gamma_0@v1_coeff
    w1_0 = gamma_0@w1_coeff
    
    u2_0 = gamma_0@u2_coeff
    v2_0 = gamma_0@v2_coeff
    w2_0 = gamma_0@w2_coeff
    
    f1_dot1 = -omega_0*v1 - u1_prime
    f2_dot1 = omega_0*u1 - 2*chi1*w1 - v1_prime
    f3_dot1 = 2*chi1*v1 - w1_prime
    f4_dot1 = u1_0
    f5_dot1 = v1_0
    f6_dot1 = w1_0 + 1
    
    f1_dot2 = -omega_0*v2 - u2_prime
    f2_dot2 = omega_0*u2 - 2*chi2*w2 - v2_prime
    f3_dot2 = 2*chi2*v2 - w2_prime
    f4_dot2 = u2_0
    f5_dot2 = v2_0
    f6_dot2 = w2_0 + 1
    
    fk1 = np.hstack((f1_dot1,f2_dot1,f3_dot1,f4_dot1,f5_dot1,f6_dot1))
    fk2 = np.hstack((f1_dot2,f2_dot2,f3_dot2,f4_dot2,f5_dot2,f6_dot2))
    
    return np.hstack((fk1,fk2))

In [27]:
def Newton(coefficients_guess,t=t_k,eps=1e-8):
    max_iteration = 100000
    f = get_f(coefficients_guess,t)
    coefficients = coefficients_guess

    for iteration in range(max_iteration):
        print('iteration:',iteration,'f',np.max(abs(f)))
        if (abs(f)<eps).all():
            print('Converged after',iteration,'iterations')
            return coefficients
        
        Jacobian = get_Jacobian(coefficients,t)
        
        delta_coeff = linalg.solve(Jacobian,f)
        coefficients -= delta_coeff 
        
        f = get_f(coefficients,t)
        
    print('Exceeded Maximum Iterations')

In [ ]:
#Creates initial guesses for u,v,w for both dots using least square approximation
gamma = get_gamma(t_elliot)
b = np.vstack((u1_elliot,v1_elliot,w1_elliot,u2_elliot,v2_elliot,w2_elliot)).T

x = linalg.solve(gamma.T@gamma,gamma.T@b)

In [ ]:
coefficients_guess =x.flatten()

In [ ]:
time_guess = gamma@x

u1_guess = time_guess[:,0]
v1_guess = time_guess[:,1]
w1_guess = time_guess[:,2]

u2_guess = time_guess[:,3]
v2_guess = time_guess[:,4]
w2_guess = time_guess[:,5]

In [ ]:
I = np.eye(3)
mat = I - np.outer(r_hat_dot1,r_hat_dot1)
der = np.exp(1j*omega_k*t_k[:,np.newaxis])

In [ ]:
der_d_vec = der*d_vector[:,np.newaxis,np.newaxis]
der_d_vec = np.transpose(der_d_vec,axes=(1,0,2))

result = np.dot(mat,(der_d_vec))
result = np.transpose(result,axes=(1,0,2))

der_chi_res = np.dot(d_vector,result)
print(d_vector.shape,result.shape)
print(der_chi_res.shape)

In [ ]:
print(t_k.size,6*omega_k.size)
print(degrees_of_freedom_remaining)

In [ ]:
#Solve system using Scipy Optimize Package
with timer():
    coeff = Newton(coefficients_guess,t=t_k,eps=1e-2)

In [ ]:
u1_coeff = coeff[:num_coefficients]
v1_coeff = coeff[num_coefficients:2*num_coefficients]
w1_coeff = coeff[2*num_coefficients:3*num_coefficients]

u2_coeff = coeff[3*num_coefficients:4*num_coefficients]
v2_coeff = coeff[4*num_coefficients:5*num_coefficients]
w2_coeff = coeff[5*num_coefficients:]

u1 = gamma@u1_coeff
v1 = gamma@v1_coeff
w1 = gamma@w1_coeff

u2 = gamma@u2_coeff
v2 = gamma@v2_coeff
w2 = gamma@w2_coeff

In [ ]:
compare_frequency_spectra_solutions(u1_guess,u1_elliot,t_elliot,360,364)

In [ ]:
plt.figure()
plt.plot(t_elliot,u1,label='Chad solution u1')
plt.plot(t_elliot,u1_elliot,label='Elliot solution u1')
plt.xlim(0,0.05)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,v1,label='Chad solution v1')
plt.plot(t_elliot,v1_elliot,label='Elliot solution v1')
plt.xlim(0,0.05)
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,w1,label='Chad solution w1')
plt.plot(t_elliot,w1_elliot,label='Elliot solution w1')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(t_elliot,np.sqrt(u1**2 + v1**2),label='chad')
plt.plot(t_elliot,np.sqrt(u1_elliot**2 + v1_elliot**2),label='elliot')
plt.legend()
plt.show()